In [ ]:
!pip install -q \
  sentence-transformers \
  faiss-cpu \
  pypdf \
  transformers \
  accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 21.0 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader

def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

raw_text = load_pdf("main_notes.pdf")

print("Characters extracted:", len(raw_text))
print("\n--- SAMPLE TEXT ---\n")
print(raw_text[:1000])


Characters extracted: 373593

--- SAMPLE TEXT ---

CS229 Lecture Notes
Andrew Ng and Tengyu Ma
June 11, 2023
Contents
I Supervised learning 5
1 Linear regression 8
1.1 LMS algorithm . . . . . . . . . . . . . . . . . . . . . . . . . . 9
1.2 The normal equations . . . . . . . . . . . . . . . . . . . . . . . 13
1.2.1 Matrix derivatives . . . . . . . . . . . . . . . . . . . . . 13
1.2.2 Least squares revisited . . . . . . . . . . . . . . . . . . 14
1.3 Probabilistic interpretation . . . . . . . . . . . . . . . . . . . . 15
1.4 Locally weighted linear regression (optional reading) . . . . . . 17
2 Classiﬁcation and logistic regression 20
2.1 Logistic regression . . . . . . . . . . . . . . . . . . . . . . . . 20
2.2 Digression: the perceptron learning algorithm . . . . . . . . . 23
2.3 Multi-class classiﬁcation . . . . . . . . . . . . . . . . . . . . . 24
2.4 Another algorithm for maximizing ℓ(θ) . . . . . . . . . . . . . 27
3 Generalized linear models 29
3.1 The exponential family . . . . .

In [ ]:
import re

def clean_text(text):
    # Remove excessive newlines
    text = re.sub(r'\n+', '\n', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

cleaned_text = clean_text(raw_text)

print("Cleaned characters:", len(cleaned_text))
print("\n--- CLEANED SAMPLE ---\n")
print(cleaned_text[:1000])


Cleaned characters: 373574

--- CLEANED SAMPLE ---

CS229 Lecture Notes Andrew Ng and Tengyu Ma June 11, 2023 Contents I Supervised learning 5 1 Linear regression 8 1.1 LMS algorithm . . . . . . . . . . . . . . . . . . . . . . . . . . 9 1.2 The normal equations . . . . . . . . . . . . . . . . . . . . . . . 13 1.2.1 Matrix derivatives . . . . . . . . . . . . . . . . . . . . . 13 1.2.2 Least squares revisited . . . . . . . . . . . . . . . . . . 14 1.3 Probabilistic interpretation . . . . . . . . . . . . . . . . . . . . 15 1.4 Locally weighted linear regression (optional reading) . . . . . . 17 2 Classiﬁcation and logistic regression 20 2.1 Logistic regression . . . . . . . . . . . . . . . . . . . . . . . . 20 2.2 Digression: the perceptron learning algorithm . . . . . . . . . 23 2.3 Multi-class classiﬁcation . . . . . . . . . . . . . . . . . . . . . 24 2.4 Another algorithm for maximizing ℓ(θ) . . . . . . . . . . . . . 27 3 Generalized linear models 29 3.1 The exponential family . . . . 

In [ ]:
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []

    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i + chunk_size])
        if len(chunk.strip()) > 0:
            chunks.append(chunk)

    return chunks

chunks = chunk_text(cleaned_text)

print("Total chunks:", len(chunks))
print("\n--- SAMPLE CHUNK ---\n")
print(chunks[10][:1000])


Total chunks: 266

--- SAMPLE CHUNK ---

resent functions/hypotheses h in a computer. As an initial choice, let’s say we decide to approximate y as a linear function of x: hθ(x) = θ0 + θ1x1 + θ2x2 Here, the θi’s are the parameters (also called weights) parameterizing the space of linear functions mapping from X to Y. When there is no risk of 8 9 confusion, we will drop the θ subscript in hθ(x), and write it more simply as h(x). To simplify our notation, we also introduce the convention of letting x0 = 1 (this is the intercept term), so that h(x) = d∑ i=0 θixi = θTx, where on the right-hand side above we are viewing θ and x both as vectors, and here d is the number of input variables (not counting x0). Now, given a training set, how do we pick, or learn, the parameters θ? One reasonable method seems to be to make h(x) close to y, at least for the training examples we have. To formalize this, we will deﬁne a function that measures, for each value of the θ’s, how close the h(x(i))’s are t

In [ ]:
def chunk_text(text, chunk_size=150, overlap=30):
    words = text.split()
    chunks = []

    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i + chunk_size])
        if len(chunk.strip()) > 0:
            chunks.append(chunk)

    return chunks

chunks = chunk_text(cleaned_text)

print("Total chunks:", len(chunks))
print("\n--- SAMPLE CHUNK ---\n")
print(chunks[20][:800])

Total chunks: 553

--- SAMPLE CHUNK ---

is the living area of the i-th house in the training set, and x(i) 2 is its number of bedrooms. (In general, when designing a learning problem, it will be up to you to decide what features to choose, so if you are out in Portland gathering housing data, you might also decide to include other features such as whether each house has a ﬁreplace, the number of bathrooms, and so on. We’ll say more about feature selection later, but for now let’s take the features as given.) To perform supervised learning, we must decide how we’re going to rep- resent functions/hypotheses h in a computer. As an initial choice, let’s say we decide to approximate y as a linear function of x: hθ(x) = θ0 + θ1x1 + θ2x2 Here, the θi’s are the parameters (also called weights) parameterizing the space of linear function


In [ ]:
def chunk_text(text, chunk_size=120, overlap=30):
    words = text.split()
    chunks = []

    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i + chunk_size])
        if len(chunk.strip()) > 0:
            chunks.append(chunk)

    return chunks

chunks = chunk_text(cleaned_text)

print("Total chunks:", len(chunks))
print("\n--- SAMPLE CHUNK ---\n")
print(chunks[30][:800])

Total chunks: 738

--- SAMPLE CHUNK ---

−y(i))2. If you’ve seen linear regression before, you may recognize this as the familiar least-squares cost function that gives rise to the ordinary least squares regression model. Whether or not you have seen it previously, let’s keep going, and we’ll eventually show this to be a special case of a much broader family of algorithms. 1.1 LMS algorithm We want to choose θ so as to minimize J(θ). To do so, let’s use a search algorithm that starts with some “initial guess” for θ, and that repeatedly changes θ to make J(θ) smaller, until hopefully we converge to a value of θ that minimizes J(θ). Speciﬁcally, let’s consider the gradient descent algorithm, which starts with some initial θ, and repeatedly


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(
    chunks,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embedding shape:", embeddings.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Embedding shape: (738, 384)


In [ ]:
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Vectors indexed:", index.ntotal)

Vectors indexed: 738


In [ ]:
def retrieve(query, k=5):
    q_emb = embedder.encode([query])
    distances, indices = index.search(q_emb, k)
    return [chunks[i] for i in indices[0]]

results = retrieve("What is gradient descent?")

for i, r in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(r[:500])


--- Result 1 ---
n, { θj := θj + α ( y(i) −hθ(x(i)) ) x(i) j , (for every j) (1.2) } } By grouping the updates of the coordinates into an update of the vector θ, we can rewrite update (1.2) in a slightly more succinct way: θ:= θ+ α ( y(i) −hθ(x(i)) ) x(i) In this algorithm, we repeatedly run through the training set, and each time we encounter a training example, we update the parameters according to the gradient of the error with respect to that single training example only. This algorithm is called stochastic 

--- Result 2 ---
This algorithm is called stochastic gradient descent (also incremental gradient descent). Whereas batch gradient descent has to scan through the entire training set before taking a single step—a costly operation if n is large—stochastic gradient descent can start making progress right away, and 13 continues to make progress with each example it looks at. Often, stochastic gradient descent gets θ “close” to the minimum much faster than batch gra- dient descent

In [ ]:
from transformers import pipeline

generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=300
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def answer_question(question, k=5):
    # 1. Retrieve relevant chunks
    context_chunks = retrieve(question, k=k)
    context_text = "\n\n".join(context_chunks)

    # 2. Build strict RAG prompt
    prompt = f"""
You are a Stanford CS229 teaching assistant.
Answer the question using ONLY the lecture notes below.
If the answer is not contained in the notes, say exactly:
"I don't know based on the provided notes."

Lecture Notes:
{context_text}

Question:
{question}
"""

    # 3. Generate answer
    response = generator(prompt)
    return response[0]["generated_text"]

In [ ]:
answer_question("What is gradient descent?")

Token indices sequence length is longer than the specified maximum sequence length for this model (1023 > 512). Running this sequence through the model will result in indexing errors


'This algorithm is called stochastic gradient descent (also incremental gradient descent).'

In [ ]:
answer_question('What is a null value?')

'0'

In [ ]:
answer_question('what is confusion matrix?')

'd-by-d permutation matrix'

In [ ]:
answer_question('What is Random Forest?')

"I don't know based on the provided notes."

In [ ]:
answer_question('What is the page number for gradient descent?')

'13'

In [ ]:
!pip install -q streamlit pyngrok

In [ ]:
answer_question('What is gradient descent?')

'This algorithm is called stochastic gradient descent (also incremental gradient descent).'